<a href="https://colab.research.google.com/github/narendra974/GAN/blob/main/vae_auto_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
import numpy as np



In [7]:
batch_size = 100
original_dim = 28*28
latent_dim = 2
intermediate_dim = 256
nb_epoch = 5
epsilon_std = 1.0

In [8]:
def Sampling(args):
  z_mean, z_log_var  = args
  epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0)
  return z_mean + K.exp(z_log_var/2) * epsilon


x = Input(shape=original_dim, name="input")
h = Dense(intermediate_dim, activation='relu', name='encoding')(x)
z_mean = Dense(latent_dim, name='mean')(h)
z_variance = Dense(latent_dim, name='log-variance')(h)
z=Lambda(Sampling, output_shape=(latent_dim, ))([z_mean, z_variance])
encoder = Model(x, [z_mean, z_variance, z], name="encoder")

In [9]:
input_decoder = Input(shape=(latent_dim, ), name='decoder_input')
decoder_h = Dense(intermediate_dim, activation='relu', name='decoder_h')(input_decoder)
x_decoded = Dense(original_dim, activation='sigmoid', name='flat_decoded')(decoder_h)
decoder = Model(input_decoder, x_decoded, name='decoder')

In [10]:
from keras.metrics import binary_crossentropy

def vae_loss(x, x_decoded_mean, z_log_var, z_mean, original_dim):
  xent_loss = original_dim * binary_crossentropy(x,x_decoded_mean)
  kl_loss = original_dim * K.sum(1 + z_log_var - K.squre(z_mean)-K.exp(z_log_var), axis=-1)
  return xent_loss + kl_loss

output_combined =decoder(encoder(x)[2])
vae = Model(x, output_combined)
vae.summary()

vae.compile(optimizer='rmsprop', loss=vae_loss)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 784)]             0         
                                                                 
 encoder (Functional)        [(None, 2),               201988    
                              (None, 2),                         
                              (100, 2)]                          
                                                                 
 decoder (Functional)        (None, 784)               202256    
                                                                 
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [13]:
(x_train,y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

vae.fit(x_train, x_train, shuffle=True, epochs=nb_epoch, batch_size=batch_size, validation_data=(x_test, x_test), verbose=1)

Epoch 1/5


TypeError: ignored